# Find repeated strings in different text files

**The problem**: we have two text files containing names of companies. The problem is that the company names are sometimes spelled differently. For example, in one of them we have _"Western Digital"_ and in the other _"Western Digital Corp."_.

We'll use Pandas to find those company names repeated and normalize them. We're also using an excellent Python library for string comparison: [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy).

In [1]:
!pip install fuzzywuzzy

You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import itertools
from fuzzywuzzy import fuzz, process

/Users/santiagobasulto/.virtualenvs/ds-classes/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Read both files into its own DataFrame

In [3]:
df1 = pd.read_csv('test_CSV_1.csv')
df2 = pd.read_csv('test_CSV_2.csv')

Each file has 200/300 companies, not too large.

In [4]:
df1.size, df2.size

(266, 368)

And we can extract company names as regular np.arrays:

In [5]:
csv_1 = df1['CLIENT'].values
csv_2 = df2['Firm Name'].values

In [6]:
csv_1[:10]

array(['Adobe Systems, Inc.', 'Adventist Health', 'AECOM',
       'Aerojet Rockedyne Holdings (GenCorp)',
       'Alameda-Contra Costa Transit District',
       'Alaska Community Foundation',
       'Alaska Retirement Management Board', 'Alexander & Baldwin, Inc.',
       'Allergan, Inc.', 'Alyeska Pipeline Service Company'], dtype=object)

In [7]:
csv_2[:10]

array(['AAA Northern California, Nevada & Utah Auto Exchange',
       'ACCO Engineered Systems', 'Adams County Retirement Plan',
       'Adidas America, Inc.', 'Adobe Systems, Inc.',
       'Advanced Micro Devices, Inc.', 'AECOM Technology Corporation',
       'Aera Energy LLC', 'Aerojet Rocketdyne Holdings, Inc.',
       'Agilent Technologies, Inc.'], dtype=object)

In this example you can already notice repeated "similar" companies: `"AECOM"` in `csv_1` and `"AECOM Technology Corporation"` in `csv_2`.

To do our fuzzy comparison, we'll need to first build a "cartensian product" with all our companies. It'll be an "expensive" operation, and it'll create a big array with all the occurrences of `csv_2`, **per company in `csv_1`**. `csv_1` has 266 companies and `csv_2` has 368, the total will be: $266 * 368 = 97888$

![Cartesian product](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Cartesian_Product_qtl1.svg/440px-Cartesian_Product_qtl1.svg.png)

We create the cartesian product:

In [8]:
companies = list(itertools.product(csv_1, csv_2))

And as you can see we have 97888 companies:

In [9]:
len(companies)

97888

In [10]:
df = pd.DataFrame(companies, columns=['CSV 1', 'CSV 2'])

This how our product looks like:

In [11]:
df.head()

,CSV 1,CSV 2
0,"Adobe Systems, Inc.","AAA Northern California, Nevada & Utah Auto Ex..."
1,"Adobe Systems, Inc.",ACCO Engineered Systems
2,"Adobe Systems, Inc.",Adams County Retirement Plan
3,"Adobe Systems, Inc.","Adidas America, Inc."
4,"Adobe Systems, Inc.","Adobe Systems, Inc."


We'll now apply the `fuzz.partial_ratio` function from the `fuzzywuzzy` package (there are others, check [their docs](https://github.com/seatgeek/fuzzywuzzy))

In [12]:
df['Ratio Score'] = df.apply(lambda row: fuzz.partial_ratio(*row), axis=1)

And we have a score for reach occurrence:

In [13]:
df.head()

,CSV 1,CSV 2,Ratio Score
0,"Adobe Systems, Inc.","AAA Northern California, Nevada & Utah Auto Ex...",21
1,"Adobe Systems, Inc.",ACCO Engineered Systems,62
2,"Adobe Systems, Inc.",Adams County Retirement Plan,33
3,"Adobe Systems, Inc.","Adidas America, Inc.",47
4,"Adobe Systems, Inc.","Adobe Systems, Inc.",100


As you can see, _"Adobe"_ has a 100 score, which makes sense, what about _"AECOM"_?

"AECOM" in csv_1 and "AECOM Technology Corporation" in c

In [14]:
df[(df['CSV 1'] == 'AECOM') & (df['CSV 2'] == "AECOM Technology Corporation")]

,CSV 1,CSV 2,Ratio Score
742,AECOM,AECOM Technology Corporation,100


As you can see, it has a perfect score.

Now we'll take a look at all those companies that provided a good match. I'll set our cutoff score to 90, you can try different values:

In [15]:
df[(df['Ratio Score'] > 90)]

,CSV 1,CSV 2,Ratio Score
4,"Adobe Systems, Inc.","Adobe Systems, Inc.",100
742,AECOM,AECOM Technology Corporation,100
1484,Alameda-Contra Costa Transit District,Alameda-Contra Costa Transit District,100
3697,Amazon,"Amazon.com Holdings, Inc.",100
4435,Amgen Inc.,Amgen Inc.,100
5175,"Apple, Inc.","Apple, Inc.",100
5544,Applied Materials,"Applied Materials, Inc.",100
6286,Arizona Public Safety Personnel Retirement System,Arizona Public Safety Personnel Retirement System,100
7023,Arizona State Retirement System,Arizona State Retirement System,100
8865,Asante Health System,Asante Health System & Subsidiaries,100


The problem we have here is that we still have repeated names with perfect matches, for example _"Apple, Inc."_ with _"Apple, Inc."_. We can obviate those:

In [16]:
df[(df['Ratio Score'] > 90) & (df['CSV 1'] != df['CSV 2'])]

,CSV 1,CSV 2,Ratio Score
742,AECOM,AECOM Technology Corporation,100
3697,Amazon,"Amazon.com Holdings, Inc.",100
5544,Applied Materials,"Applied Materials, Inc.",100
8865,Asante Health System,Asante Health System & Subsidiaries,100
9605,Automobile Club of Southern California (AAA of...,Automobile Club of Southern California,100
9974,Avaya,Avaya Inc.,100
11081,Avnet,"Avnet, Inc.",100
12189,Bechtel,"Bechtel Group, Inc.",100
13299,Boise Cascade,Boise Cascade Holdings LLC,100
16252,California Savings Plus,California Savings Plus Program,100


And now it looks much better 😊, check a few occurrences, for example `"Wells Fargo" | "Wells Fargo & Company"` or `"Western Digital" | "Western Digital Corp."`

Finally, we can export our results to Excel if you prefer to use spreadsheets instead of Python (for now 😉):

In [19]:
df.to_excel('output.xlsx')